In [10]:
import os
import warnings
from dotenv import load_dotenv
import pandas as pd
from langchain.chains import ConversationChain
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [2]:
warnings.filterwarnings('ignore')

In [4]:
load_dotenv()
groq_api_key = os.getenv('GROQ_API_KEY')

In [5]:
llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-8b-8192")

In [31]:
df = pd.read_csv('Data.csv')

In [32]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [9]:
from langchain.chains import LLMChain

In [16]:
prompt = ChatPromptTemplate.from_template(
    "What is the best Marketing_Channel that produces \
    new {New_Customers}? in short 2-3 sentences"
)

In [14]:
chain = LLMChain(llm=llm, prompt=prompt)

In [17]:
new_customer = 20
print(chain.run(new_customer))

The million-dollar question!

The best marketing channel that produces new customers (also known as "new 20") can vary depending on your industry, target audience, and marketing strategy. However, based on various studies and industry reports, here are some of the most effective marketing channels that can help you acquire new customers:

1. **Email Marketing**: Email marketing is a powerful channel for acquiring new customers. According to a study by Campaign Monitor, email marketing has an average return on investment (ROI) of 4400%.
2. **Social Media Advertising**: Social media platforms like Facebook, Instagram, and LinkedIn offer powerful advertising options that can help you reach new customers. According to a study by Hootsuite, social media advertising has an average ROI of 150%.
3. **Content Marketing**: Creating high-quality, relevant, and valuable content can help you attract new customers. According to a study by Content Marketing Institute, content marketing has an average

## SimpleSequentialChain

In [18]:
from langchain.chains import SimpleSequentialChain

In [19]:
llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-8b-8192")

In [20]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [21]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [22]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [24]:
product = 'Men Face Wash'
print(overall_simple_chain.run(product))



> Entering new SimpleSequentialChain chain...
Naming a company can be a challenging but exciting task. Here are some suggestions for a company that makes men's face wash:

1. **FaceForge**: A play on the word "forge," implying strength and resilience, which is perfect for a men's grooming product.
2. **ManeMatters**: A clever name that references the importance of men's grooming, with a nod to the word "matters" to emphasize the significance of taking care of one's skin.
3. **Beard & Beyond**: This name acknowledges the popular beard trend while also hinting at the idea that the company's products can help men take care of their skin beyond just their beard.
4. **Razor's Edge**: A name that references the sharp, clean shave that many men strive for, while also implying a sense of precision and effectiveness.
5. **CleanCut**: A simple, straightforward name that conveys the idea of a clean, refreshing face wash that leaves skin feeling smooth and revitalized.
6. **Grit & Glow**: A name

## SequentialChain

In [25]:
from langchain.chains import SequentialChain

In [26]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [27]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [28]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [30]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [33]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [34]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': 'Here is the translation:\n\n"I find the taste mediocre. The mousse doesn\'t hold up, it\'s weird. I buy the same ones in stores and the taste is much better...\nOld lot or counterfeit?!"\n\nThe reviewer is expressing disappointment with the product, saying that the taste is mediocre and the mousse doesn\'t hold up well. They compare it to buying the same product in stores, which has a better taste. The final question "Vieux lot ou contrefaçon ?!" is a rhetorical question, implying that the product is either old or counterfeit.',
 'summary': 'The reviewer is disappointed with the product, finding the taste mediocre and the mousse texture poor, and suspects that it may be an old or counterfeit product.',
 'followup_message': 'Voici ma réponse :\n\n« Je suis déçu par ce produit, car le goût est

## Router Chain

In [35]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [36]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [37]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [38]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [39]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [40]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [41]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [42]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [43]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation in the context of thermodynamics?'}
> Finished chain.


"Black body radiation is a fundamental concept in thermodynamics, and it's a fascinating topic.\n\nIn the context of thermodynamics, black body radiation refers to the thermal radiation emitted by an idealized object, known as a blackbody, when it is heated to a high temperature. A blackbody is an idealized object that absorbs all the radiation that falls on it, without reflecting or transmitting any of it.\n\nWhen a blackbody is heated, it emits radiation across a wide range of wavelengths, from the ultraviolet to the infrared. This radiation is known as blackbody radiation. The key characteristic of blackbody radiation is that it is independent of the properties of the emitting object; it only depends on the temperature of the object.\n\nIn other words, the radiation emitted by a blackbody is solely determined by its temperature, and not by its composition, shape, or size. This is in contrast to other types of radiation, such as the radiation emitted by a light bulb or a star, which 

In [45]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
physics: {'input': "Why does every cell in our body contain DNA? (assuming it's because DNA contains the genetic instructions for the development and function of all living organisms)"}
> Finished chain.


"What a great question!\n\nYou're absolutely right that DNA contains the genetic instructions for the development and function of all living organisms. And that's precisely why every cell in our body contains DNA.\n\nHere's the thing: DNA is like a blueprint or a set of instructions that contains the genetic information necessary for the development and function of an organism. It's like a recipe book for building and maintaining life.\n\nEvery cell in our body contains DNA because it's essential for the cell to know how to function properly. Think of it like a computer program: just as a computer needs software to run, a cell needs DNA to know how to perform its specific functions.\n\nIn other words, DNA contains the instructions for the cell to:\n\n1. Grow and divide\n2. Respond to its environment\n3. Regulate its internal processes\n4. Maintain its structure and function\n\nWithout DNA, a cell would be like a computer without software – it wouldn't know how to do its job!\n\nSo, to 